In [15]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts import PromptTemplate


def create_quiz(questions):
    quiz = ""
    for i, q in enumerate(questions):
        quiz += f"{i+1}. {q['question']}\n"
        for j, a in enumerate(q["answers"]):
            quiz += f"    {chr(97+j)}. {a['answer']}\n"
        quiz += "\n"
    return quiz


function = {
    "name": "create_quiz",
    "description": (
        "function that takes a list of questions and answers and returns a"
        " quiz"
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,
).bind(
    functions=[function],
    function_call={
        "name": "create_quiz",
    },
)

prompt = PromptTemplate.from_template("Make a quiz about {city}?")

chain = prompt | llm

response = chain.invoke({"city": "Rome"})

response = response.additional_kwargs["function_call"]["arguments"]

In [16]:
import json

questions = []
for question in json.loads(response)["questions"]:
    q = {"question": question["question"], "answers": []}
    for answer in question["answers"]:
        q["answers"].append(
            {"answer": answer["answer"], "correct": answer["correct"]}
        )
    questions.append(q)
quiz = create_quiz(questions)
print(quiz)

1. What year was Rome founded?
    a. 753 BC
    b. 476 AD
    c. 1492 AD

2. Who was the first emperor of Rome?
    a. Julius Caesar
    b. Augustus
    c. Nero

3. What famous structure in Rome was built by the ancient Romans and is still standing today?
    a. Colosseum
    b. Pantheon
    c. Trevi Fountain

4. Which volcano erupted and buried the city of Pompeii near Rome?
    a. Mount Vesuvius
    b. Mount Etna
    c. Mount Olympus

5. Who was the legendary founder of Rome according to Roman mythology?
    a. Romulus
    b. Remus
    c. Caesar


